In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

# Load data
file_path = "dataset/sales_4.csv"
df = pd.read_csv(file_path)

# Preprocess data
df['date'] = pd.to_datetime(df['date'])
df = df.groupby(['date', 'store_id', 'product_name'])['product_count'].sum().reset_index()

# Pivot data to create time series for each store and product
pivoted = df.pivot_table(
    index=['store_id', 'product_name'],
    columns='date',
    values='product_count',
    fill_value=0
)
pivoted = pivoted.sort_index(axis=1)  # Sort by date

# Normalize data
data_array = pivoted.to_numpy()
data_array = (data_array - data_array.mean(axis=1, keepdims=True)) / data_array.std(axis=1, keepdims=True)

# Graph structure (Example: Fully connected graph)
store_ids = pivoted.index.get_level_values('store_id').unique()
num_stores = len(store_ids)
edges = [(i, j) for i in range(num_stores) for j in range(num_stores) if i != j]
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# Convert data to PyTorch tensors
x = torch.tensor(data_array, dtype=torch.float)  # Node features (time series data)
num_time_steps = x.size(1)

# Define ST-GNN model
class STGNN(nn.Module):
    def __init__(self, num_nodes, num_features, hidden_dim, out_channels):
        super(STGNN, self).__init__()
        self.gcn1 = GCNConv(num_features, hidden_dim)
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        self.gru = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, out_channels)

    def forward(self, x, edge_index):
        # Spatial component
        x = self.gcn1(x, edge_index).relu()
        x = self.gcn2(x, edge_index).relu()

        # Ensure reshaping matches dimensions
        num_nodes = x.size(0) // num_time_steps
        x = x.view(num_nodes, num_time_steps, -1)  # Reshape for GRU

        # Temporal component
        x, _ = self.gru(x)

        # Prediction
        x = self.fc(x[:, -1, :])  # Use last time step for prediction
        return x


# Initialize model
num_features = num_time_steps
hidden_dim = 64
out_channels = 1
model = STGNN(num_stores, num_features, hidden_dim, out_channels)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(x, edge_index)
    loss = criterion(output, x[:, -1])  # Predict the next time step
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Save the trained model
torch.save(model.state_dict(), "stgnn_demand_forecast.pth")


RuntimeError: shape '[7, 1425, -1]' is invalid for input of size 696128